In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import requests
from io import StringIO
import yfinance as yf
import random
import math

In [ ]:
# Fundamentals of Stochastic Modeling 

# 1. Brownian Motion

# Brownian motion (or a Wiener process) is any process which has zero mean, linearly increasing variance as a function of time, and no autocorrelation. Thus it is any process in which the sequence of random variables all follow the same distribution. One common representation is the following

# Two-Point Wiener Processes

def twoPointDist(n):
    x = random.random()
    if x <= 0.5:
        return -1/np.sqrt(n)
    else:
        return 1/np.sqrt(n)
    
def binarySamplePath(n):
    xData172 = np.linspace(0,1,n)
    yData172 = []
    xStep = 0

    for x in xData172:
        dx = twoPointDist(100)
        xStep = xStep + dx
        yData172.append(xStep)

    return xData172, yData172

# Now build a dictionary with the keys labeling the sample path and the value storing the sample path in question. 

brownDct = {}
tIndex = np.linspace(0,1,100)

for n in range(1000):
    wienerPath = binarySamplePath(100)[1]
    brownDct[n+1] = wienerPath

wienerDf = pd.DataFrame(brownDct,tIndex)

wienerDf.plot(legend=False)
plt.show()

In [ ]:
# This can also be done with a Gaussian distribution, which is more common

def gaussianDist(mu,sigma):
    return np.random.normal(mu,sigma)

def gaussianWienerProc(n,t1):
    dt = t1/n

    tArray = np.linspace(dt,t1,n)
    xArray = [0]
    x = xArray[0]

    for t in tArray:
        x = x + gaussianDist(0,np.sqrt(dt))
        xArray.append(x)
        continue

    tArray = np.concat((np.array([0]),tArray))

    return tArray, xArray


gaussWienerDct = {}

for n in range(100):
    wProc = gaussianWienerProc(1000,1)
    gaussWienerDct[n+1] = wProc[1]

tIndex = wProc[0]
gaussWienerDf = pd.DataFrame(gaussWienerDct,tIndex)

gaussWienerDf.plot(legend=False)
plt.show()

std = gaussWienerDf.var(axis=1)

std.plot()
plt.show()

# Notice that the variance increases linearly with time, as expected.

In [ ]:
# What happens if we try to calculate the derivative of a Wiener process? This can be done with either the binary wiener process, or the gaussian one. We use the gaussian one for the sake of completeness. Notice the derivative diverges as the partitions become finer and finer. This suggests that the notion of derivative here doesn't necessarily make sense (although it probably does in the path space).

# Now try to evaluate the derivative of this function at t = 0.5 by numerically calculating the derivative for successively smaller areas around 0.5.

deriv174 = []
hVals174 = []

for n in range(1000,10010,10):
    yData174 = gaussianWienerProc(n,1)[1]
    index = math.floor(n/2)
    yDiff = (yData174[index+1]-yData174[index])/(1/n)
    deriv174.append(yDiff)
    hVals174.append(1/n)

plt.plot(hVals174,deriv174)

# Importantly, notice that the derivative diverges in a wild way as one tries to calculate the derivative in terms of GIVEN values, so we need to do something more general.